In [93]:
import pandas as pd
import pickle
# Load dataset
df = pd.read_csv("/content/sample_data/matches300.csv", sep=',')
df.fillna(0, inplace=True)
df = df[df['winner'] != 0.5]

# Compute average ACS and KD per team
team1_stats = df[['team1', 'AVG_KD1', 'AVG_ACS1']].rename(columns={'team1': 'team', 'AVG_KD1': 'KD', 'AVG_ACS1': 'ACS'})
team2_stats = df[['team2', 'AVG_KD2', 'AVG_ACS2']].rename(columns={'team2': 'team', 'AVG_KD2': 'KD', 'AVG_ACS2': 'ACS'})
team_stats = pd.concat([team1_stats, team2_stats]).groupby('team').mean().reset_index()

# Merge team statistics into original dataset
df = df.merge(team_stats, left_on='team1', right_on='team', how='left').rename(columns={'KD': 'team1_KD', 'ACS': 'team1_ACS'}).drop(columns=['team'])
df = df.merge(team_stats, left_on='team2', right_on='team', how='left').rename(columns={'KD': 'team2_KD', 'ACS': 'team2_ACS'}).drop(columns=['team'])


In [95]:
from sklearn.model_selection import train_test_split

# Split data to train and test

input_features = ['team1_KD', 'team2_KD', 'team1_ACS', 'team2_ACS', 'map1', 'map2', 'map3','patch','league']
target_feature = 'winner'

X_train, X_test, y_train, y_test = train_test_split(df[input_features], df[target_feature], test_size=0.2, random_state=4)


In [96]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score


# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [97]:
from xgboost import XGBClassifier
import joblib

# Initialize the XGBoost model
model_xgb = XGBClassifier(eval_metric='mlogloss')

# Train the model
model_xgb.fit(X_train, y_train)

# Make predictions
y_pred_xgb = model_xgb.predict(X_test)

# Print results
print("XGBoost Classifier:")
print(classification_report(y_test, y_pred_xgb))
print(f"Accuracy: {accuracy_score(y_test, y_pred_xgb):.4f}")

joblib.dump(model_xgb, "xgboost_classifier.pkl")

XGBoost Classifier:
              precision    recall  f1-score   support

         0.0       0.73      0.70      0.71        23
         1.0       0.81      0.83      0.82        36

    accuracy                           0.78        59
   macro avg       0.77      0.76      0.77        59
weighted avg       0.78      0.78      0.78        59

Accuracy: 0.7797


['xgboost_classifier.pkl']

In [92]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Define the model
model = Sequential([
    Dense(32, activation='relu', input_shape=(len(input_features),)),  # Input layer
    Dropout(0.2),  # Prevent overfitting
    Dense(16, activation='relu'),  # Hidden layer
    Dense(1, activation='sigmoid')  # Output layer (sigmoid for classification)
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))

# Evaluate model performance
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

model.save("sequential_model.h5")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 8s 50ms/step - accuracy: 0.4614 - loss: 0.7180 - val_accuracy: 0.6271 - val_loss: 0.6687
Epoch 2/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.5523 - loss: 0.6771 - val_accuracy: 0.6949 - val_loss: 0.6379
Epoch 3/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.6061 - loss: 0.6639 - val_accuracy: 0.7458 - val_loss: 0.6108
Epoch 4/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.6940 - loss: 0.6109 - val_accuracy: 0.7881 - val_loss: 0.5849
Epoch 5/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - accuracy: 0.6913 - loss: 0.5996 - val_accuracy: 0.8136 - val_loss: 0.5549
Epoch 6/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.7220 - loss: 0.5681 - val_accuracy: 0.8220 - val_loss: 0.5291
Epoch 7/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.7446 - loss: 0.5375 - val_accuracy: 0.8136 - val_loss: 0.5041
Epoch 8/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.7859 - loss: 0.5051 - val_accuracy: 0.7712 - v

Test Accuracy: 0.7797


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# Logistic Regression Model
model_lr = Pipeline([
    ('scale', StandardScaler()),  # Normalize numerical data
    ('lr', LogisticRegression(max_iter=1000))
])

model_lr.fit(X_train, y_train)
y_pred_lr = model_lr.predict(X_test)

# Print results
print("Logistic Regression:")
print(classification_report(y_test, y_pred_lr))
print(f"Accuracy: {accuracy_score(y_test, y_pred_lr):.4f}")

with open("logistic_regression.pkl", "wb") as file:
    pickle.dump(model_lr, file)

Logistic Regression:
              precision    recall  f1-score   support

         0.0       0.70      0.78      0.74        45
         1.0       0.85      0.79      0.82        73

    accuracy                           0.79       118
   macro avg       0.78      0.79      0.78       118
weighted avg       0.79      0.79      0.79       118

Accuracy: 0.7881


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Random Forest Model
model_rf = Pipeline([
    ('scale', StandardScaler()),
    ('rf', RandomForestClassifier(n_estimators=50, random_state=5))
])

model_rf.fit(X_train, y_train)
y_pred_rf = model_rf.predict(X_test)

# Print results
print("\nRandom Forest Classifier:")
print(classification_report(y_test, y_pred_rf))
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")

with open("random_forest_classifier.pkl", "wb") as file:
    pickle.dump(model_rf, file)


Random Forest Classifier:
              precision    recall  f1-score   support

         0.0       0.79      0.65      0.71        23
         1.0       0.80      0.89      0.84        36

    accuracy                           0.80        59
   macro avg       0.79      0.77      0.78        59
weighted avg       0.80      0.80      0.79        59

Accuracy: 0.7966


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Linear Regression Model
model_lrr = LinearRegression()
model_lrr.fit(X_train, y_train)
y_pred_lrr = model_lrr.predict(X_test)
mae = mean_absolute_error(y_test, y_pred_lrr)
mse = mean_squared_error(y_test, y_pred_lrr)

# Print results
print("\nLinear Regression:")
print(f'Mean Absolute Error: {mae}',f'Mean Squared Error: {mse}')


Linear Regression:
Mean Absolute Error: 0.3252818627374903 Mean Squared Error: 0.15581965877229229
